In [5]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import requests
from bs4 import BeautifulSoup

In [12]:
url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
result = requests.get(url).text
soup = BeautifulSoup(result,'lxml')
table = soup.find('table',{'class':'wikitable plainrowheaders'})

In [14]:
# it shows all 'tr' in the url from the first one to the last one
#[1::1] it means skipping the first finding, and then slice it one by one same as [1:]
#we did this because we dont want to import the header name
for items in soup.find('table', class_='wikitable').find_all('tr')[1::1]:
    data = items.find_all(['th','td'])
    try:
        country = data[0].a.text
        title = data[1].a.text
        name = data[1].a.find_next_sibling().text
    except IndexError:pass
    print("{}|{}|{}".format(country,title,name))

None


In [109]:
table = soup.find('table', class_='wikitable')
#data is in form of tr>>td + filling>> td>>td + filling>>td>>td+filling>>td tr>>tr and so on
#this is all considered as one item in a loop so the row is an item
df = pd.DataFrame(columns=['postal_code','borough','neighbourhood'])
# it shows all 'tr' in the url from the first one to the last one
#[1::1] it means skipping the first finding, and then slice it one by one same as [1:]
#we did this because we dont want to import the header name
#Each cell is considered as <td>filling + </td>
for row in table.find_all('tr')[1::1]:
    cells = []
    for cell in table.find_all('td'):
        #using the text will remove td, and then do strip to remove any spaces
        cell_value = cell.text.strip()
        #now we added each cell value in a list
        cells.append(cell_value)
#adding cells values to the dataframe by jumping 2 steps from each start
df['postal_code'] = cells[::3]
df['borough'] = cells[1::3]
df['neighbourhood'] = cells[2::3]
df.head()

,postal_code,borough,neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"


In [130]:
#removing rows where borough is not assigned
df = df[df['borough']!= 'Not assigned']
#checking if there is ant not assigned value, we found it none
df.isin(['Not assigned']).any()

postal_code      False
borough          False
neighbourhood    False
dtype: bool

In [178]:
#joining neighbourhoods for different boroughs
dfjoin = df.groupby('postal_code')['neighbourhood'].apply(lambda x: ','.join(x))
dfjoin.reset_index()
df2 = dfjoin.reset_index()
df2.rename(columns = {'neighbourhood':'neighbourhood_joined'}, inplace= True)
df2.head()
df_merged = pd.merge(df, df2)
df_merged = df_merged.drop(['neighbourhood'], axis=1)
df_merged.drop_duplicates(inplace=True)
df_merged.head()

,postal_code,borough,neighbourhood_joined
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


In [179]:
df.nunique(), df.shape

(postal_code      103
 borough          10 
 neighbourhood    99 
 dtype: int64,
 (103, 3))

In [181]:
df_merged.nunique(), df2.shape

(postal_code             103
 borough                 10 
 neighbourhood_joined    99 
 dtype: int64,
 (103, 2))

In [182]:
geo= pd.read_csv('Geospatial_Coordinates.csv')
geo.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [190]:
final_df = pd.merge(df_merged, geo, left_on='postal_code', right_on='Postal Code')
#we make sure that both columns are the same
(final_df['postal_code'] != final_df['Postal Code']).sum().any()

False

In [193]:
final_df.drop(['Postal Code'], axis=1, inplace=True)
final_df.head()

,postal_code,borough,neighbourhood_joined,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494


In [201]:
df_toronto = final_df[final_df.borough.str.contains('Toronto')]
df_toronto.head()

,postal_code,borough,neighbourhood_joined,Latitude,Longitude
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494
9,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937
15,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418
19,M4E,East Toronto,The Beaches,43.676357,-79.293031


In [203]:
df_toronto.nunique()

postal_code             39
borough                 4 
neighbourhood_joined    39
Latitude                39
Longitude               34
dtype: int64

In [202]:
df_North_York = final_df[final_df.borough.str.contains('North York')]
df_North_York.head()

,postal_code,borough,neighbourhood_joined,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
7,M3B,North York,Don Mills,43.745906,-79.352188
10,M6B,North York,Glencairn,43.709577,-79.445073


In [204]:
df_North_York.nunique()

postal_code             24
borough                 1 
neighbourhood_joined    20
Latitude                24
Longitude               21
dtype: int64